# Lasso Regression with Hourly Data

In [ ]:
# Imports
import requests
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from scipy import linalg
from sklearn import linear_model
from sklearn.linear_model import Ridge
from pytrends.request import TrendReq
import os.path


**Helper Functions**

In [ ]:
def save_dict_df(dictex, keys_filename="keys.txt", data_folder="hourly_stock_data"):
    for key, val in dictex.items():
        val.to_csv("./"+data_folder+"/data_{}.csv".format(str(key)))

    with open(keys_filename, "w") as f: #saving keys to file
        f.write(str(list(dictex.keys())))

def load_dict_df(keys_filename="keys.txt", data_folder="hourly_stock_data"):
    """Reading data from keys"""
    with open(keys_filename, "r") as f:
        keys = eval(f.read())

    dictex = {}    
    for key in keys:
        dictex[key] = pd.read_csv("./"+data_folder+"/data_{}.csv".format(str(key)), index_col=0)

    return dictex

In [ ]:
def getTrends(company_name, df_trends):
    # Filters the trends
    company_keywords =  [x for x in list(df_trends.columns.values) if company_name in x]
    return df_trends[company_keywords].copy()
    

In [ ]:
def getJointTimes(df_stock, df_trends): 
    stock_times = df_stock['times']
    trends_times = list(df_trends.index)
    return list(set(stock_times) & set(trends_times)) 

In [ ]:
# Define Company Keyterms and Symbols
companies = ['tesla', 'facebook', 'microsoft', 'amazon', 'google', 'uber', 'lyft', 'apple', 'snap']
key_terms = ['report', 'good', 'bad', 'up', 'down', 'stock']
company_symbol = ['TSLA', 'FB', 'MSFT', 'AMZN', 'GOOGL', 'UBER', 'LYFT', 'AAPL', 'SNAP']



In [ ]:
#Create Key Word List
kw_list = []
for c_name in companies:
    for k in key_terms:
        kw_list.append(c_name + " " + k)

**Gather Google trends data**

In [ ]:
# Get Hourly trends data from pytrends
# If there is a server 500 error, try changing the dates to this past week! 

def getHourlyTrends(company_symbol, pull_data=False, trends_df_filename="hourly_pytrends.csv"):
    if not pull_data and os.path.isfile(trends_df_filename):
        trends_df = pd.read_csv(trends_df_filename, index_col="date")
    else :
        df = pd.DataFrame()
        data = {}
        pytrends = TrendReq(hl='en-US', tz=360)
        for kw in kw_list:
            print(kw)
            df_temp = pytrends.get_historical_interest([kw], year_start=2019, month_start=11, day_start=28, hour_start=0, year_end=2019, month_end=12, day_end=4, hour_end=23)
            if 'isPartial' in df_temp.columns: 
                df_temp = df_temp.drop(['isPartial'], axis=1)
            data[kw] = df_temp
        for kw in kw_list:
            if df.empty:
                df = data[kw]
            else:
                df = df.join(data[kw])
                
        print("finished kw forloop")
        trends_df = df
        df.to_csv(trends_df_filename)
    return trends_df
        

In [ ]:
df_trends = getHourlyTrends(company_symbol, pull_data=True)

In [ ]:
# Inspect Data
df_trends.head()

**Gather Stock Data: Alpha Vantage**

In [ ]:
# Gather Stock Data for each company and return dataframe 

def getIntradayInfoDict(company_symbol, pull_data=False, stock_df_filename="hourly_stock_keys.txt") :
    if not pull_data and os.path.isfile(stock_df_filename):
        dict_stocks = load_dict_df(keys_filename=stock_df_filename);
    else :
        dict_stocks = {}
        ts = 'TIME_SERIES_INTRADAY'#'TIME_SERIES_DAILY'
        interval = '30min'
        api_key = 'OK92GXML7JXOOOFD' 
        outputsize = 'full' # compact= 100 results, full= all data (5 days?)
        for i, symbol in enumerate(company_symbol):
            link = 'https://www.alphavantage.co/query?function={}&symbol={}&interval={}&apikey={}&outputsize={}'\
                .format(ts, symbol, interval, api_key, outputsize)
            request = requests.get(link)
            data = json.loads(request.text)
            if "Time Series (30min)" not in data:
                print("data limit reached. sleeping")
                time.sleep(70)
                link = 'https://www.alphavantage.co/query?function={}&symbol={}&interval={}&apikey={}&outputsize={}'\
                    .format(ts, symbol, interval, api_key, outputsize)
                request = requests.get(link)
                data = json.loads(request.text)
            stock_data = json.dumps(data["Time Series (30min)"])
            df = pd.read_json(stock_data).transpose()
            cols = ['1. open', '2. high', '3. low', '4. close', '5. volume']
            df = df[cols]
            df.reset_index(level=0, inplace=True)
            df.columns = ['times', 'open', 'high', 'low', 'close', 'volume']
            dict_stocks[companies[i]] = df
        save_dict_df(dict_stocks, keys_filename=stock_df_filename)
    return dict_stocks

In [ ]:
df_stocks = getIntradayInfoDict(company_symbol, pull_data=False)

In [ ]:
#reverse df rows
df_stocks['tesla'] = df_stocks['tesla'].iloc[::-1]

In [ ]:
df_stocks['tesla'].tail()

**Merge Trend and Stock Data**

In [ ]:
df_trends_stocks = {}
def cleanAndJoinData():
    for s,c in zip(company_symbol, companies):
        print(c)
        company_names = [x for x in list(df.columns.values) if c in x]
        df_temp_trends = df[company_names]
        
        # line up indexes 
        stock_times = list(df_stocks[s].times)
        trends_times = list(df_temp_trends.index)
        joint_times = list(set(stock_times) & set(trends_times)) 
        
        print(joint_times)
        df_temp_stocks = df_stocks[s].loc[df_stocks[s]['times'].isin(joint_times)]
        df_temp_stocks = df_temp_stocks.reset_index()
        df_temp_stocks = df_temp_stocks.iloc[::-1]
        df_temp_trends = df_temp_trends.loc[df_temp_trends.index.isin(joint_times)]
        df_temp_trends = df_temp_trends.reset_index()
        df_temp_trends.columns = ['_'.join(x.split()) for x in list(df_temp_trends.columns) if len(x) > 1]
        df_trends_stocks[c] = df_temp_stocks.join(df_temp_trends,lsuffix='_left', rsuffix='_right')
cleanAndJoinData()

**Function for calculating new predictors**

In [ ]:
def getMovingAvgAndStdDev(col_name, num_days, df_all_data):
    # Moving average and stdev past X days
    col_movingAvg = []
    col_stdev = []
    
    num_days_to_average = 10
    for i in range(len(df_all_data)):
        sum_to_avg = 0
        nums = []
        num_to_avg = min(num_days_to_average, len(df_all_data) - i) - 1
        for j in range(1, num_to_avg):
            sum_to_avg += df_all_data[col][i + j]
            nums.append(df_all_data[col][i + j])
        avg = sum_to_avg / (num_to_avg if num_to_avg > 0 else 1)
        stdev = np.std((nums if nums else [0]))

        col_movingAvg.append(avg)
        col_stdev.append(stdev)
    return col_movingAvg, col_stdev

**Add Additional Predictors to model**

In [ ]:
def getAllParamCols(df_all_data):
    cols = list(df_all_data.columns)
    cols.remove('open')
    cols.remove('close')
    cols.remove('high')
    cols.remove('low')
    cols.remove('times')
    cols.remove('date')
    cols.remove('index')
    cols.remove('volume')
    cols.remove('profit')
    cols.remove('mid')
    cols.remove('volume_movingAvg')
    cols.remove('volume_stdev')
    cols.remove('profit_movingAvg')
    cols.remove('profit_stdev')
    cols.remove('mid_movingAvg')
    cols.remove('mid_stdev')
    return cols

In [ ]:
# Add predictors
for company in df_trends_stocks: 
    # Calculate Profit For Each Company in Dict
    (df_trends_stocks[company])['profit'] = (df_trends_stocks[company])['open']-(df_trends_stocks[company])['close']
    (df_trends_stocks[company])['mid'] = ((df_trends_stocks[company])['high']+(df_trends_stocks[company])['low'])/2
    # Calculate Moving Averages and Standard Deviation for Stock Data
    # For Profit
    num_days_to_average = 10
    col = 'profit'
    col_movingAvg, col_stdev = getMovingAvgAndStdDev(col, num_days_to_average, df_trends_stocks[company])
    (df_trends_stocks[company])[col+'_movingAvg'] = col_movingAvg
    (df_trends_stocks[company])[col+'_stdev'] = col_stdev
    col_prev = [(df_trends_stocks[company])[col][i+1] for i in range(len((df_trends_stocks[company])) - 1)]
    col_prev.append(0) # Append this so we can have 0 padding
    df_trends_stocks[company][col+'_prev'] = col_prev
    # For Mid
    col = 'mid'
    col_movingAvg, col_stdev = getMovingAvgAndStdDev(col, num_days_to_average, df_trends_stocks[company])
    (df_trends_stocks[company])[col+'_movingAvg'] = col_movingAvg
    (df_trends_stocks[company])[col+'_stdev'] = col_stdev
    col_prev = [(df_trends_stocks[company])[col][i+1] for i in range(len((df_trends_stocks[company])) - 1)]
    col_prev.append(0) # Append this so we can have 0 padding
    df_trends_stocks[company][col+'_prev'] = col_prev
    # For Volume
    col = 'volume'
    col_movingAvg, col_stdev = getMovingAvgAndStdDev(col, num_days_to_average, df_trends_stocks[company])
    (df_trends_stocks[company])[col+'_movingAvg'] = col_movingAvg
    (df_trends_stocks[company])[col+'_stdev'] = col_stdev
    cols = getAllParamCols(df_trends_stocks[company])
    col_prev = [(df_trends_stocks[company])[col][i+1] for i in range(len((df_trends_stocks[company])) - 1)]
    col_prev.append(0) # Append this so we can have 0 padding
    df_trends_stocks[company][col+'_prev'] = col_prev
    
    # Calculate Increment Over time
    for col in cols: 
        # Prev 
        col_prev = [(df_trends_stocks[company])[col][i+1] for i in range(len((df_trends_stocks[company])) - 1)]
        col_prev.append(0) # Append this so we can have 0 padding
        df_all_data[col+'_prev'] = col_prev
        
        col_movingAvg, col_stdev = getMovingAvgAndStdDev(col, num_days_to_average, df_trends_stocks[company])
        
        (df_trends_stocks[company])[col+'_movingAvg'] = col_movingAvg
        (df_trends_stocks[company])[col+'_stdev'] = col_stdev
    # Reverse index order
    df_trends_stocks[company] = df_trends_stocks[company].iloc[::-1] 

**LASSO Regression Helper Functions**

In [ ]:
# Methods for LASSO Regression
def getBestAlphaLASSORegression(y_col, parameter_cols, df_all_data_train):
    split = int(df_all_data_train.shape[0]/2)
    df_all_data_test = df_all_data_train[split:]
    df_all_data_train = df_all_data_train[:split]
    
    X = df_all_data_train[parameter_cols]
    y = df_all_data_train[y_col]
    alpha = []
    MSE_train = []
    MSE_test = []
    for i in range(90, 10000, 10):
        clf = linear_model.Lasso(alpha=i)
        clf.fit(X, y) 
        alpha.append(i)
        MSE_train.append(mean_squared_error(clf.predict(df_all_data_train[cols]), df_all_data_train[y_col]))
        MSE_test.append(mean_squared_error(clf.predict(df_all_data_test[cols]), df_all_data_test[y_col]))

    bestAlpha = alpha[MSE_test.index(min(MSE_test))]
    
    print("alpha: "+str(bestAlpha))
    print("Training error = "+str(mean_squared_error(clf.predict(df_all_data_train[cols]), df_all_data_train[y_col])))
    print("Testing error = "+str(mean_squared_error(clf.predict(df_all_data_test[cols]), df_all_data_test[y_col])))
    print()
    return bestAlpha

In [ ]:
# Define Functions For Plotting LASSO Regression
def plotLASSO(company, alpha, label, predictors, df): 
    # Separate Data
    split = int(df.shape[0]/2)
    df_train = df[:split]
    df_test = df[split:]
    X_train = df_train[predictors]
    Y_train = df_train[label]
    X_test = df_test[predictors]
    Y_test = df_test[label]
    Y_labels = df[label]
    # Initialize Model w/ Optimal Alpha
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(X_train, Y_train) 
    param_dict = dict(zip(clf.coef_, predictors))
    print("Parameter Estimates w/ LASSO : ",param_dict)
    # Make Predictions using optimal alpha value
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    # Plot test and train predictions against true labels
    
    fig1, ax1 = plt.subplots()
    ax1.plot(range(split), y_pred_train, color='r')
    ax1.plot(range(split, len(Y_labels)),y_pred_test, color='b')
    ax1.plot(range(len(Y_labels)),Y_labels, color='g')
    ax1.set_xlabel('Time in Hours')
    ax1.set_ylabel(label)
    ax1.set_title(label + ' Predictions with LASSO Regression for '+ company)

**Run regression and view dropped features for each company**

In [ ]:
def columnsAll(company, label): 
    cols = df_trends_stocks[company].columns
    cols = list(cols)
    cols.remove('times')
    cols.remove('index')
    cols.remove('date')
    cols.remove(label)
    return cols

def columnsTrends(company, label): 
    cols = [x for x in list(df_trends_stocks[company].columns) if company in x]
    return cols

def columnsAverage(company, label): 
    cols = [x for x in list(df_trends_stocks[company].columns) if company not in x]
    cols.remove(label)
    cols.remove('times')
    cols.remove('index')
    cols.remove('date')
    cols.remove('open')
    cols.remove('volume')
    cols.remove('high')
    cols.remove('low')
    cols.remove('close')
    cols.remove('profit')
    return cols

# Plot Predictions w/ All Predictors

In [ ]:
keys = ['All Predictors ', 'Just Trends ','Just Stock Data']
MSE = {key: None for key in keys}
for company in companies: 
    # Prepare column list
    cols = columnsAll(company, 'mid') 
    # Find best alpha for LASSO Regression
    alpha = getBestAlphaLASSORegression('mid',cols,df_trends_stocks[company])
    # Plot Predictions by True Labels
    plotLASSO(company, alpha, 'mid', cols, df_trends_stocks[company])
    # Plot MSE train and test for LASSO Regression

# Plot Predictions for Just Trends Data

In [ ]:
for company in companies: 
    # Prepare column list
    cols = columnsTrends(company, 'mid') 
    # Find best alpha for LASSO Regression
    alpha = getBestAlphaLASSORegression('mid',cols,df_trends_stocks[company])
    # Plot Predictions by True Labels
    plotLASSO(company, alpha, 'mid', cols, df_trends_stocks[company])
    # Plot MSE train and test for LASSO Regression

# Plot Predictions for Just Stock Data

In [ ]:
for company in companies: 
    # Prepare column list
    cols = columnsAverage(company, 'mid') 
    # Find best alpha for LASSO Regression
    alpha = getBestAlphaLASSORegression('mid',cols,df_trends_stocks[company])
    # Plot Predictions by True Labels
    plotLASSO(company, alpha, 'mid', cols, df_trends_stocks[company])
    # Plot MSE train and test for LASSO Regression